In [72]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt 
import seaborn as sn
from sklearn import preprocessing, svm
from sklearn import svm, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
%matplotlib inline

In [73]:
test = pd.read_csv("adult-1.test", header=None, names=['Age', 'Workclass', 'FNLWGT','Education', 'Education#', 'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Capital_Gain', 'Capital_Loss', 'Hours_Per_Week', 'Native_Country', '50K'], skiprows=1)
train = pd.read_csv("adult-2.data", header=None, names=['Age', 'Workclass', 'FNLWGT','Education', 'Education#', 'Marital_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Capital_Gain', 'Capital_Loss', 'Hours_Per_Week', 'Native_Country', '50K'])

In [74]:
train.head(6)

,Age,Workclass,FNLWGT,Education,Education#,Marital_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_Per_Week,Native_Country,50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [75]:
test.head(6)

,Age,Workclass,FNLWGT,Education,Education#,Marital_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_Per_Week,Native_Country,50K
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.


In [76]:
df = pd.concat([train, test], axis = 0)
dff = df
k = df

In [77]:
df['50K'] = df['50K'].apply(lambda x: 1 if x==' >50K' else 0)

In [78]:
for col in df.columns:
    if type(df[col][0]) == str:
        print("Working on " + col)
        df[col] = df[col].apply(lambda val: val.replace(" ",""))

In [79]:
df.replace(' ?', np.nan, inplace=True)

In [80]:
df.head(3)

,Age,Workclass,FNLWGT,Education,Education#,Marital_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_Per_Week,Native_Country,50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0


In [81]:
df = pd.concat([df, pd.get_dummies(df['Workclass'],prefix='Workclass',prefix_sep=':')], axis=1)
df.drop('Workclass',axis=1,inplace=True)

df = pd.concat([df, pd.get_dummies(df['Marital_Status'],prefix='Marital_Status',prefix_sep=':')], axis=1)
df.drop('Marital_Status',axis=1,inplace=True)

df = pd.concat([df, pd.get_dummies(df['Occupation'],prefix='Occupation',prefix_sep=':')], axis=1)
df.drop('Occupation',axis=1,inplace=True)

df = pd.concat([df, pd.get_dummies(df['Relationship'],prefix='Relationship',prefix_sep=':')], axis=1)
df.drop('Relationship',axis=1,inplace=True)

df = pd.concat([df, pd.get_dummies(df['Race'],prefix='Race',prefix_sep=':')], axis=1)
df.drop('Race',axis=1,inplace=True)

df = pd.concat([df, pd.get_dummies(df['Sex'],prefix='Sex',prefix_sep=':')], axis=1)
df.drop('Sex',axis=1,inplace=True)

df = pd.concat([df, pd.get_dummies(df['Native_Country'],prefix='Native_Country',prefix_sep=':')], axis=1)
df.drop('Native_Country',axis=1,inplace=True)

df.drop('Education', axis=1,inplace=True)

df.head()

,Age,FNLWGT,Education#,Capital_Gain,Capital_Loss,Hours_Per_Week,50K,Workclass: Federal-gov,Workclass: Local-gov,Workclass: Never-worked,...,Native_Country: Portugal,Native_Country: Puerto-Rico,Native_Country: Scotland,Native_Country: South,Native_Country: Taiwan,Native_Country: Thailand,Native_Country: Trinadad&Tobago,Native_Country: United-States,Native_Country: Vietnam,Native_Country: Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
dff.head(3)

,Age,Workclass,FNLWGT,Education,Education#,Marital_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_Per_Week,Native_Country,50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0


In [82]:
dff['50K'] = dff['50K'].apply(lambda x: 1 if x==' >50K.' else 0)

In [84]:
dff.replace(' ?', np.nan, inplace=True)

In [85]:
dff.fillna(' 0', inplace=True)

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
X = np.array(df.drop(['50K'], 1))
y = np.array(df['50K'])
X = preprocessing.scale(X)
y = np.array(df['50K'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



In [90]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export

clf_tree = DecisionTreeClassifier( max_depth = 4 )

clf_tree.fit( X_train, y_train )
tree_predict = clf_tree.predict( X_test )
metrics.accuracy_score( y_test, tree_predict )

print(confusion_matrix(y_test,tree_predict))
print(classification_report(y_test,tree_predict))
DTA = accuracy_score(y_test, tree_predict)
print("The Accuracy for Decision Tree Model is {}".format(DTA))

[[8042  195]
 [1164  368]]
              precision    recall  f1-score   support

           0       0.87      0.98      0.92      8237
           1       0.65      0.24      0.35      1532

    accuracy                           0.86      9769
   macro avg       0.76      0.61      0.64      9769
weighted avg       0.84      0.86      0.83      9769

The Accuracy for Decision Tree Model is 0.8608864776333299


In [92]:
from sklearn import preprocessing,neighbors
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

clf = neighbors.KNeighborsClassifier()

clf.fit(X_train, y_train)

knnpre = clf.predict(X_test)

##########Results

print(confusion_matrix(y_test,knnpre))
print(classification_report(y_test,knnpre))
KKNA = accuracy_score(y_test, knnpre)
print("The Accuracy for KNN is {}".format(KKNA))

[[7651  586]
 [1040  492]]
              precision    recall  f1-score   support

           0       0.88      0.93      0.90      8237
           1       0.46      0.32      0.38      1532

    accuracy                           0.83      9769
   macro avg       0.67      0.63      0.64      9769
weighted avg       0.81      0.83      0.82      9769

The Accuracy for KNN is 0.8335551233493704


In [94]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score

lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
y_lmpre = lm.predict(X_test)

ylm = np.around(y_lmpre)

lm.score(X_train, y_train)
lm.coef_
lm.intercept_

##########Results

print(confusion_matrix(y_test,ylm))
print(classification_report(y_test,ylm))

LA = accuracy_score(y_test, ylm)
print("The Accuracy for Linear Regression is {}".format(LA))

[[8130  107]
 [1369  163]]
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      8237
           1       0.60      0.11      0.18      1532

    accuracy                           0.85      9769
   macro avg       0.73      0.55      0.55      9769
weighted avg       0.82      0.85      0.80      9769

The Accuracy for Linear Regression is 0.8489098167673252
